<a href="https://colab.research.google.com/github/PencilMario/stable-diffusion-webui/blob/master/waifu_squarelizer_shared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WAIFU SQUARELIZER 
repo : https://github.com/IzumiSatoshi/waifu-squarelizer  
The image in the input folder is cropped like this using facial recognition, and output to the output folder.  
The "square_size_ratio" parameter determines the size of the square to be cropped relative to the short side of the image. For example, if a vertical image is processed at 0.5, it will be cropped by a square half the size of the image width.  
![waifu-sq.jpg](https://raw.githubusercontent.com/IzumiSatoshi/waifu-squarelizer/main/images/waifu-sq.jpg)

In [ ]:
# mount google drive if you want
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# installation
!pip install openmim
!mim install mmcv-full
!mim install mmdet
!mim install mmpose

!pip install anime-face-detector

In [ ]:
# imports
import cv2
from anime_face_detector import create_detector
from google.colab.patches import cv2_imshow
import glob
import os

In [ ]:
# load model
detector = create_detector('yolov3')

In [ ]:
# main
def squarelize(image, square_size_ratio = 1):
  preds = detector(image)

  if len(preds) == 0:
    return None 

  left = preds[0]['bbox'][0]
  bottom = preds[0]['bbox'][1]
  right = preds[0]['bbox'][2]
  top = preds[0]['bbox'][3]
  face_x = int((left + right) / 2)
  face_y = int((top + bottom) / 2)
  height, width = image.shape[:2]
  
  square_size = None
  if height > width:
    # portrait
    square_size = int(width * square_size_ratio)
  else:
    # landscape
    square_size = int(height * square_size_ratio)
  
  square_size_half = square_size // 2
  crop_x = None
  crop_y = None
  
  if face_x < square_size_half:
    crop_x = square_size_half
  elif face_x < width - square_size_half:
    crop_x = face_x
  else:
    crop_x = width - square_size_half
  
  if face_y < square_size_half:
    crop_y = square_size_half   
  elif face_y < height - (square_size_half):
    crop_y = face_y
  else:
    crop_y = height - square_size_half
  
  sq_image = image[
      crop_y - square_size_half : crop_y + square_size_half,
      crop_x - square_size_half : crop_x + square_size_half
  ]

  return sq_image


input_dir = "/content/drive/MyDrive/test/input"#@param{type:"string"}
output_dir = "/content/drive/MyDrive/test/output"#@param{type:"string"}
output_extension = "jpg"#@param{type:"string"}
square_size_ratio = 1#@param {type:"slider", min:0, max:1, step:0.1}

if not os.path.exists(output_dir):
  raise ValueError("output_dir is not exist")

paths = glob.glob(input_dir + "/*")
paths_len = len(paths)
error_list = []

for i, path in enumerate(paths):
  print(f"{i}/{paths_len} : {path}")
  basename = os.path.splitext(os.path.basename(path))[0]

  image = cv2.imread(path)
  sq_image = squarelize(image, square_size_ratio)
  
  # Skip if face does not exist
  if sq_image is None:
    print("Could not recognize the face in this image")
    error_list.append(path)
    continue

  sq_image = cv2.resize(sq_image, dsize=(512, 512), interpolation = cv2.INTER_AREA)
  cv2.imwrite(f"{output_dir}/{basename}.{output_extension}", sq_image)

print("error list = ", error_list)
print("done. enjoy!")